<a href="https://colab.research.google.com/github/kregier/AudioLanguageClassifer/blob/main/SampleModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is this notebook?
This is a notebook to load a few audio files and load the VGGish model. The idea is to make sure the model loads and runs before moving it to the larger notebook.

In [1]:
# Set up the environment
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import IPython.display as ipd
import librosa
import librosa.display

import os
import random

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

print("All set up!")

All set up!


# Load the data
- Connect to google drive
- Load a few sample audio files


In [2]:
# Set up the data import using Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

# Change working directory
%cd /content/gdrive/My Drive/Kaggle
!ls

/content/gdrive/My Drive/Kaggle
data  kaggle.json  reading-passage.txt	recordings  speakers_all.csv


In [61]:
meta = pd.read_csv('speakers_all.csv')

#Filenames
x_train = ['afrikaans1', 'mandarin3','french38']
x_test = ['spanish94', 'lao2']

In [62]:
# Sample audio files
afrikaans = 'recordings/recordings/afrikaans1.mp3'
mandarin = 'recordings/recordings/mandarin3.mp3'
spanish = 'recordings/recordings/spanish94.mp3'
french = 'recordings/recordings/french38.mp3'
lao = 'recordings/recordings/lao2.mp3'

SAMP_RATE = 16000

afrikaans_raw, sr = librosa.load(afrikaans, sr=SAMP_RATE)
mandarin_raw, sr = librosa.load(mandarin, sr=SAMP_RATE)
spanish_raw, sr = librosa.load(spanish, sr=SAMP_RATE)
french_raw, sr = librosa.load(french, sr=SAMP_RATE)
lao_raw, sr = librosa.load(lao, sr=SAMP_RATE)

### Format features for testing and training sets

In [63]:
# Results in np arrays of different lengths, since the audio files are different lengths
x_train_features = np.asarray([afrikaans_raw, mandarin_raw, french_raw])
# print(x_train_features.shape)
# print(type(x_train_features))

In [64]:
x_test_features = np.asarray([spanish_raw, lao_raw])
# print(x_test_features.shape)
# print(type(x_test_features))

In [65]:
# Segment the files into 10s arrays to have consistent input dimensions
def get_10s(audio, sr):
  """ Load an audio file and get the first 10 seconds.
  Arguments: audio - the audio file; sr = sampling rate of the file
  Returns: first 10s of audio file.
  """
  beginning = audio[0:10*sr]
  return beginning

In [25]:
#x_train_beg = np.asarray([get_10s(i, SAMP_RATE) for i in x_train_features])
#print(x_train_beg.shape)
#print(type(x_train_beg))

(4, 160000)
<class 'numpy.ndarray'>


In [66]:
# Scale audio to fall between [-1, 1]
def normalize(audio):
  norm = audio/max(audio)
  return norm

In [67]:
x_train_norm = np.asarray([normalize(get_10s(i, SAMP_RATE)) for i in x_train_features])
#x_train_norm = np.asarray([normalize(i) for i in x_train_beg])
print(x_train_norm.shape)
#print(type(x_train_norm))
#print(type(x_train_norm[0]))
#print(type(x_train_norm[0][0]))

# Reshape x_train_norm to have shape (None, 4, 16000)
x_train_norm = x_train_norm[None, :,:]

print(x_train_norm.shape)
#print(type(x_train_norm))
#print(type(x_train_norm[0]))
#print(type(x_train_norm[0][0]))
#print(type(x_train_norm[0][0][0]))

(3, 160000)
(1, 3, 160000)


In [68]:
# Format x_test array
x_test_norm = np.asarray([normalize(get_10s(i, SAMP_RATE)) for i in x_test_features])
print(x_test_norm.shape)

x_test_norm = x_test_norm[None, :,:]
print(x_test_norm.shape)
# print(type(x_test_norm))
# print(type(x_test_norm[0]))
# print(type(x_test_norm[0][0]))
# Need to reshape  this array!

(2, 160000)
(1, 2, 160000)


In [69]:
# Format label arrays
def gender_str_to_int(labels):
  y_label = []
  for name in labels:
    idx = meta[meta.filename == name].index
    gender = meta.loc[idx, 'sex'].values[0]
    if gender == 'male':
      y_label.append(1)
    else: y_label.append(0)
  return np.asarray(y_label)

In [72]:
y_train_label = gender_str_to_int(x_train)
print(type(y_train_label))
print(y_train_label.shape)

# Convert to tensor shape for model input
y_train_label = y_train_label[None, :]
print(type(y_train_label))
print(y_train_label.shape)

<class 'numpy.ndarray'>
(3,)
<class 'numpy.ndarray'>
(1, 3)


In [73]:
y_test_label = gender_str_to_int(x_test)
print(type(y_test_label))
print(y_test_label.shape)

# Convert to tensor shape for model input
y_test_label = y_test_label[None, :]
print(type(y_test_label))
print(y_test_label.shape)

<class 'numpy.ndarray'>
(2,)
<class 'numpy.ndarray'>
(1, 2)


# Load the pre-trained VGGish model from Tensorflow Hub

In [78]:
# Link to the model on TFHub
hub_url = 'https://tfhub.dev/google/vggish/1'

# Load the model as a Keras model
vggish_model = hub.KerasLayer(hub_url)
#vggish_model.summary()

## Run sample audio through the model and examine the embedding

In [79]:
for i in range(len(x_train_norm)):
  vggish_embed = vggish_model(x_train_norm[0][i])
  print(vggish_embed.shape, vggish_embed.dtype)

(10, 128) <dtype: 'float32'>


# Embed the vggish model/embeddings into a binary gender classifier.

In [87]:
#classifier = tf.keras.Sequential([
#    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
#])

genderClf = tf.keras.models.Sequential([ #vggish_model,
                              tf.keras.layers.Dense(128, activation = 'relu', input_shape = (3, 160000)),
                              tf.keras.layers.Dense(64, activation = 'relu'),
                              tf.keras.layers.Dense(1, activation='sigmoid')
                              ])
genderClf.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])

In [88]:
# Add early stopping to train classifier model
# default is 10 epochs
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=2)

#genderClf.fit(x_train_norm[0], y_train_label[0], epochs=10) #, callbacks=[early_stopping_monitor]) #validation_split=0.25,

In [89]:
genderClf.fit(x_train_norm, y_train_label, epochs=10, callbacks=[early_stopping_monitor]) #validation_split=0.25,

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.7088 - accuracy: 0.3333WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


1/1 [==============================] - 0s 8ms/step - loss: 0.7088 - accuracy: 0.3333
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 1.3908e-06 - accuracy: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


1/1 [==============================] - 0s 4ms/step - loss: 1.3908e-06 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


1/1 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


1/1 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


...

1/1 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


1/1 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


1/1 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


1/1 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


1/1 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [92]:
y_pred = genderClf.predict(x_test_norm) #, y_test_label)
#y_pred = genderClf.predict(normalize(get_10s(spanish_raw, SAMP_RATE)), y_test_label)

In [94]:
print(y_pred)

print(y_test_label)

[[[0.97188115]
  [0.42228356]]]
[[1 0]]


In [96]:
genderClf.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 3, 128)            20480128  
_________________________________________________________________
dense_13 (Dense)             (None, 3, 64)             8256      
_________________________________________________________________
dense_14 (Dense)             (None, 3, 1)              65        
Total params: 20,488,449
Trainable params: 20,488,449
Non-trainable params: 0
_________________________________________________________________


## Confirm the types and shapes of the input vectors

In [84]:
print(type(x_train_norm))
print(x_train_norm.shape)

print(type(y_train_label))
print(y_train_label.shape)

<class 'numpy.ndarray'>
(1, 3, 160000)
<class 'numpy.ndarray'>
(1, 3)


In [85]:
print(x_test_norm.shape)
print(type(x_test_norm))

(1, 2, 160000)
<class 'numpy.ndarray'>


In [ ]:
for i in range(len(x_train_norm)):
  print(type(x_train_norm[i]))
  print(x_train_norm[i].shape)
  print(type(x_train_norm[i][0]))
  print("- - - - - - - - -")

# Next steps:
Embed the vggish model into a trainable binary classifier.

In [ ]:
# enable fine-tuning with trainable argument
#layer = hub.KerasLayer(..., trainable=True)

# Reexport the fine-tuned model

#loaded_obj = hub.load("https://tfhub.dev/...")
#hub_layer = hub.KerasLayer(loaded_obj, trainable=True, ...)

#model = keras.Sequential([..., hub_layer, ...])
#model.compile(...)
#model.fit(...)

#export_module_dir = os.path.join(os.getcwd(), "finetuned_model_export")
#tf.saved_model.save(loaded_obj, export_module_dir)

Epoch - on complete pass through the dataset

Batch size = divide dataset into smaller parts/sets

Iterations - number of batches to complete one epoch

In [ ]:
#model.fit(x, y, batch_size=n, epochs=n) # Batch size default is 32
# Do not use batch size if the data is in the form of dataset, generators, or keras.utils.Sequence instances

# Import larger sample set
- at least two batches, to see how model fits
- a least one batch of testing data

In [ ]:
meta.head()

In [ ]:
# Select 96 files at random from meta.filename
data = random.choice(meta.filename, size=96, replace=False)

idx = meta[meta.filename.isin(data)].index
df = meta.loc[idx, ['filename', 'sex']]

df.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['filename'], df['sex'], random=38, test_size=0.33)

In [ ]:
# Sample audio files
def load_data(series):
  output = []
  for file in range(len(series)):
    filepath = 'recordings/recordings/' + file + '.mp3'
    soundfile, sr = librosa.load(filepath, sr=SAMP_RATE)
    output.append(normalize(get_10s(soundfile)))
  return np.asarray(output)

In [ ]:
x_train = load_data(x_train)
print(x_train.shape)
print(type(x_train))

In [ ]:
x_test = load_data(x_test)
print(x_test.shape)
print(type(x_test))

In [ ]:
# Format label arrays
#def gender_int(labels):
#  y_label = []
#  for name in labels:
#    idx = meta[meta.filename == name].index
#    gender = meta.loc[idx, 'sex'].values[0]
#    if gender == 'male':
#      y_label.append(1)
#    else: y_label.append(0)

In [ ]:
y_train.head()